In [52]:
!pip install qiskit
!pip install qiskit_machine_learning
!pip install scikit-learn


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [53]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from qiskit_machine_learning.algorithms import QSVC
from qiskit_machine_learning.kernels import FidelityQuantumKernel
from qiskit.circuit.library import ZZFeatureMap
from qiskit.primitives import Sampler
from qiskit_algorithms.state_fidelities import ComputeUncompute
import pickle
np.random.seed(123)

In [54]:
# Load the dataset
df = pd.read_csv('cardio_train.csv', sep=';', nrows=500)
df

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,4,17474,1,156,56.0,100,60,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,711,21791,1,150,88.0,120,80,1,1,0,0,1,0
496,712,17381,1,151,117.0,160,90,2,1,0,0,0,1
497,713,20449,1,151,53.0,120,80,1,1,0,0,1,0
498,714,21197,1,162,60.0,120,80,1,1,0,0,0,0


In [55]:
# Split data into training and test sets
train, test = train_test_split(df, train_size=0.8, random_state=123)

# Select features and target variable
X_train = train[['ap_hi', 'ap_lo', 'cholesterol', 'gluc', 'smoke', 'alco']]
Y_train = train['cardio']
X_test = test[['ap_hi', 'ap_lo', 'cholesterol', 'gluc', 'smoke', 'alco']]
Y_test = test['cardio']

In [56]:
# Display data dimensions
print(f"Training features shape: {X_train.shape}")
print(f"Test features shape: {X_test.shape}")
print(f"Training target shape: {Y_train.shape}")
print(f"Test target shape: {Y_test.shape}")

Training features shape: (400, 6)
Test features shape: (100, 6)
Training target shape: (400,)
Test target shape: (100,)


In [57]:
# Create the quantum feature map
num_features = X_train.shape[1]
feature_map = ZZFeatureMap(feature_dimension=num_features, reps=2, entanglement="linear")

In [58]:
# Define Sampler and Fidelity
sampler = Sampler()
fidelity = ComputeUncompute(sampler=sampler)

C:\Users\Dell\AppData\Local\Temp\ipykernel_23948\3016593718.py:2: DeprecationWarning: The class ``qiskit.primitives.sampler.Sampler`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseSamplerV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Sampler` class is `StatevectorSampler`.
  sampler = Sampler()


In [59]:
# Define the quantum kernel
kernel = FidelityQuantumKernel(fidelity=fidelity, feature_map=feature_map)

In [60]:
# Initialize and train the QSVC model
qsvc = QSVC(quantum_kernel=kernel)
qsvc.fit(X_train, Y_train)

QSVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
     decision_function_shape='ovr', degree=3, gamma='scale', max_iter=-1,
     probability=False,
     quantum_kernel=<qiskit_machine_learning.kernels.fidelity_quantum_kernel.FidelityQuantumKernel object at 0x000001A0DCA79F70>,
     random_state=None, shrinking=True, tol=0.001, verbose=False)

In [61]:
# Evaluate the QSVC model on train and test data
qsvc_train_score = qsvc.score(X_train, Y_train)
qsvc_test_score = qsvc.score(X_test, Y_test)

In [62]:
# Print the results
print(f"QSVC classification train score: {qsvc_train_score}")
print(f"QSVC classification test score: {qsvc_test_score}")

QSVC classification train score: 0.8025
QSVC classification test score: 0.69


In [63]:
# Save the trained model using pickle
with open('qsvc_model.pkl', 'wb') as file:
    pickle.dump(qsvc, file)